## Project goal: forecasting the probability individuals recidivated year 1

## Data description:
#### The Challenge uses data on roughly 26,000 individuals from the State of Georgia released from Georgia prisons on discretionary parole to the custody of the Georgia Department of Community Supervision (GDCS) for the purpose of post-incarceration supervision between January 1, 2013 and December 31, 2015. This dataset is split into two sets, training and test. We used a 70/30 split, indicating that 70% of the data is in the training dataset and 30% in the test dataset. The training dataset includes the four dichotomous dependent variables measuring if an individual recidivated in the three-year follow-up period (yes/no) as well as recidivated by time period (year 1, year 2, or year 3). Recidivism is measured as an arrest for a new felony or misdemeanor crime within three years of the supervision start date. The test dataset does not include the four dependent variables. The initial test dataset will include all individuals selected in the 30% test dataset. After the first Challenge period (forecasting the probability individuals recidivated year 1) concludes, a second test dataset will be released containing only those individuals that did not recidivate year 1. The same will be done after the second Challenge period. It should also be noted that the test dataset will contain variables that describe supervision activities, such as drug testing and employment. These data will not appear in the test dataset until the second Challenge period (i.e., year 2 dataset). We believe this is more reflective of practice where activities must accrue and correctional oficers must become aware prior to a recidivism event. The additional data released at the second Challenge period will not change at the third Challenge period release (i.e., year 3 dataset); they are measures of supervision activities during the entire time people were under supervision or until the date of recidivism for those arrested. The only thing that changes with the third Challenge period release is the removal of those individuals that did recidivate in year 2. 
## Data Source:
#### Both the GDCS and the Georgia Bureau of Investigation provided data.

In [4]:
import csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk

In [21]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import keras
import matplotlib.pyplot as plt
import numpy as np

In [6]:
#Load train and test data
data_train = pd.read_csv("NIJ_s_Recidivism_Challenge_Training_Dataset.csv")
data_test = pd.read_csv("NIJ_s_Recidivism_Challenge_Test_Dataset1.csv")

In [8]:
# data_train.info()

In [123]:
# data_test.info()

In [7]:
# Remove rows with null values
data_train=data_train.dropna(axis=0)
data_test=data_test.dropna(axis=0)


In [8]:
# Remove the last 4 columns (y values) from the train data and removing the id column from the test and train data
X_train=data_train[data_train.columns[1:-4]]
Y_train=data_train[data_train.columns[-3]]*1
X_test=data_test[data_test.columns[1:]]

In [11]:
X_train.shape[0]
# X_train.info()

9838

In [9]:
Y_train=data_train[data_train.columns[-3]]*1
# Y_train
Y_train.shape[0]

9838

In [188]:
Y_train[1:50]

1     0
2     0
3     0
4     1
7     1
8     0
11    1
13    0
14    0
16    0
18    1
20    0
21    1
23    0
25    0
27    0
28    0
30    1
31    0
32    0
33    0
34    0
37    0
39    0
43    0
47    1
52    0
53    1
57    0
58    0
59    1
60    0
62    0
64    0
65    0
66    0
67    0
68    1
70    1
73    0
75    0
78    1
81    1
85    1
89    0
91    0
93    0
94    1
95    1
Name: Recidivism_Arrest_Year1, dtype: int32

In [129]:
# X_test.info()
X_test.shape

(5568, 32)

In [22]:
X_train.shape

(9838, 32)

In [10]:
# change the number of columns in train data from 52 to 32 (same as the test data)  
X_train=X_train[X_test.columns]
X_train.head(3)

,Gender,Race,Age_at_Release,Residence_PUMA,Gang_Affiliated,Supervision_Risk_Score_First,Supervision_Level_First,Education_Level,Dependents,Prison_Offense,...,Prior_Conviction_Episodes_Prop,Prior_Conviction_Episodes_Drug,Prior_Conviction_Episodes_PPViolationCharges,Prior_Conviction_Episodes_DomesticViolenceCharges,Prior_Conviction_Episodes_GunCharges,Prior_Revocations_Parole,Prior_Revocations_Probation,Condition_MH_SA,Condition_Cog_Ed,Condition_Other
0,M,BLACK,43-47,16,False,3.0,Standard,At least some college,3 or more,Drug,...,2,2 or more,False,False,False,False,False,True,True,False
1,M,BLACK,33-37,16,False,6.0,Specialized,Less than HS diploma,1,Violent/Non-Sex,...,0,2 or more,True,True,True,False,False,False,False,False
2,M,BLACK,48 or older,24,False,7.0,High,At least some college,3 or more,Drug,...,1,2 or more,False,True,False,False,False,True,True,False


In [11]:
X_total=X_train.append(X_test)
# X_total


In [12]:
X_total=pd.get_dummies(X_total, columns=["Gender","Race", "Age_at_Release", "Prior_Arrest_Episodes_Felony","Prior_Arrest_Episodes_Property","Prior_Arrest_Episodes_Drug",
                                         "Prior_Conviction_Episodes_Viol","Prior_Conviction_Episodes_Prop",
                                         "Prior_Arrest_Episodes_DVCharges","Prior_Conviction_Episodes_Misd",
                                         "Prior_Conviction_Episodes_Felony","Prior_Arrest_Episodes_GunCharges", 
                                         "Prior_Arrest_Episodes_PPViolationCharges", "Gang_Affiliated", 
                                         "Supervision_Level_First", "Education_Level", "Dependents", "Prison_Offense",
                                         "Prison_Years", "Prior_Arrest_Episodes_Misd" ,"Prior_Arrest_Episodes_Violent",
                                         "Prior_Conviction_Episodes_Drug", "Prior_Conviction_Episodes_PPViolationCharges",
                                         "Prior_Conviction_Episodes_DomesticViolenceCharges",
                                         "Prior_Conviction_Episodes_GunCharges", "Prior_Revocations_Parole",
                                         "Prior_Revocations_Probation", "Condition_MH_SA", "Condition_Cog_Ed",
                                         "Condition_Other"])


In [20]:
X_total.head()

,Residence_PUMA,Supervision_Risk_Score_First,Gender_M,Race_BLACK,Race_WHITE,Age_at_Release_18-22,Age_at_Release_23-27,Age_at_Release_28-32,Age_at_Release_33-37,Age_at_Release_38-42,...,Prior_Revocations_Parole_False,Prior_Revocations_Parole_True,Prior_Revocations_Probation_False,Prior_Revocations_Probation_True,Condition_MH_SA_False,Condition_MH_SA_True,Condition_Cog_Ed_False,Condition_Cog_Ed_True,Condition_Other_False,Condition_Other_True
0,16,3.0,1,1,0,0,0,0,0,0,...,1,0,1,0,0,1,0,1,1,0
1,16,6.0,1,1,0,0,0,0,1,0,...,1,0,1,0,1,0,1,0,1,0
2,24,7.0,1,1,0,0,0,0,0,0,...,1,0,1,0,0,1,0,1,1,0
3,16,7.0,1,0,1,0,0,0,0,1,...,1,0,0,1,0,1,0,1,1,0
4,16,4.0,1,0,1,0,0,0,1,0,...,1,0,1,0,0,1,0,1,0,1


In [132]:
#X_total.to_csv(r'C:\Users\sshir\TEST.csv', index=False, header=True)
X_train.shape

(9838, 32)

In [133]:
X_total.shape

(15406, 111)

In [13]:
X_train=X_total[0:X_train.shape[0]]
X_test=X_total[X_train.shape[0]:]

X_train.shape

(9838, 111)

In [136]:
# X_train.shape
Y_train.shape

(9838,)

In [209]:
C_range = [0.001,0.01,0.1,1.,10] #SVM regulization parameters
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# train an SVM Classifier
pred=np.zeros((len(C_range),X_test.shape[0]))
i=0
for x in C_range:
    clf = SVC(C=x)
    clf.fit(X_train, Y_train)
    b =  clf.predict(X_test)
    b.reshape((-1,X_test.shape[0]))
    pred[i,]=b

In [218]:
pred[4,]

array([0., 0., 0., ..., 0., 0., 0.])

In [211]:
### Step 4: Train a Random Forest Classifier, predict the labels, measure the accuracy, and print the classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Write your code here
### Step 4: Train a Random Forest Classifier, predict the labels, measure the accuracy, and print the classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# Write your code here
RFC = RandomForestClassifier (n_estimators = 200, random_state=0)
RFC.fit(X_train, Y_train)
y_pred = RFC.predict(X_test)
#accuracy=accuracy_score(y_test,y_pred)
#print('Accuracy{}'.format(accuracy))
y_pred


array([0, 0, 0, ..., 0, 0, 0])

In [17]:
print(X_train.shape,Y_train.shape)

(9838, 111) (9838,)


In [27]:
Y_train.shape

(9838,)

In [47]:
model = models.Sequential()

# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Flatten())

model.add(layers.Dense(100, input_dim=X_train.shape[1], activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# model.add(layers.Dense(23, activation="softmax"))
# model.add(layers.Dense(2, activation="softmax"))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 100)               11200     
_________________________________________________________________
dense_11 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 51        
Total params: 16,301
Trainable params: 16,301
Non-trainable params: 0
_________________________________________________________________


In [49]:
batch_size = 128
epochs = 100

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 8854 samples, validate on 984 samples
Epoch 1/100
8854/8854 [==============================] - 2s 253us/step - loss: 0.5019 - acc: 0.7503 - val_loss: 0.5288 - val_acc: 0.7510
Epoch 2/100
8854/8854 [==============================] - 0s 36us/step - loss: 0.4895 - acc: 0.7610 - val_loss: 0.5356 - val_acc: 0.7459
Epoch 3/100
8854/8854 [==============================] - 0s 37us/step - loss: 0.4813 - acc: 0.7661 - val_loss: 0.5358 - val_acc: 0.7510
Epoch 4/100
8854/8854 [==============================] - 0s 36us/step - loss: 0.4714 - acc: 0.7697 - val_loss: 0.5384 - val_acc: 0.7470
Epoch 5/100
8854/8854 [==============================] - 0s 36us/step - loss: 0.4616 - acc: 0.7782 - val_loss: 0.5594 - val_acc: 0.7236
Epoch 6/100
8854/8854 [==============================] - 0s 36us/step - loss: 0.4556 - acc: 0.7782 - val_loss: 0.5564 - val_acc: 0.7337
Epoch 7/100
8854/8854 [==============================] - 0s 36us/step - loss: 0.4460 - acc: 0.7871 - val_loss: 0.5674 - val_acc: 0.7134


In [43]:
# make probability predictions with the model
predictions = model.predict(X_test)
predictions

array([[0.125335  ],
       [0.28994012],
       [0.22991621],
       ...,
       [0.19316395],
       [0.2680969 ],
       [0.3923691 ]], dtype=float32)

In [50]:
Accuracy = model.evaluate(X_train, Y_train)


9838/9838 [==============================] - 0s 34us/step
